In [1]:
# 기본 작업 경로 설정
import os
notebook_path = os.path.abspath("project_3_git/readme.md")
notebook_dir = os.path.dirname(notebook_path)
os.chdir(notebook_dir)

# 현재 작업 디렉토리 출력
print("Current working directory: ", os.getcwd())

Current working directory:  /mnt/e/py_data/project_3_git


In [4]:
# 텍스트 데이터 가져오기
import json

# JSON 파일에서 딕셔너리 읽기
with open('data/text_data/output_text.json', 'r') as file:
    data_loaded = json.load(file)

data_loaded['0']

['공포, 헤드폰', '아찔한 분위기네요! 🎧 어떤 음악 듣고 계신가요? 궁금해요! 😊']

In [5]:
train_x = [data_loaded[i][0] for i in data_loaded][:5000]
train_y = [data_loaded[i][1] for i in data_loaded][:5000]
test_x = [data_loaded[i][0] for i in data_loaded][5000:]
test_y = [data_loaded[i][1] for i in data_loaded][5000:]

### early stopped 10
### 모델 훈련시 적정 손실값 : 0.0001 ~ 0.0003
https://huggingface.co/docs/transformers/model_doc/t5

In [5]:
from transformers import T5TokenizerFast, T5ForConditionalGeneration, Trainer, TrainingArguments, EarlyStoppingCallback
from datasets import Dataset
from tqdm import tqdm
import subprocess
import time

name_folder = 'transfer_1_more'
server_port = '1238'
batch_size = 16
train_epochs = 100

# TensorBoard 서버 실행 (백그라운드)
subprocess.Popen(['tensorboard', f'--logdir=t5/{name_folder}', f'--port={server_port}'])

# TensorBoard 서버가 시작되기를 잠시 기다립니다
time.sleep(5)

# 이후 TensorBoard 서버에 접근할 수 있습니다
print(f"TensorBoard 서버가 http://127.0.0.1:{server_port} 에서 실행중.")

# 모델과 토크나이저 로드
tokenizer = T5TokenizerFast.from_pretrained('paust/pko-t5-base')
model = T5ForConditionalGeneration.from_pretrained('paust/pko-t5-base')

# 데이터셋 생성
dataset_train = Dataset.from_dict({'input_text': train_x,'target_text': train_y})
dataset_test = Dataset.from_dict({'input_text': test_x,'target_text': test_y})



# 데이터 전처리 함수 정의
def preprocess_function(examples):
    model_inputs = tokenizer(examples['input_text'], max_length=128, truncation=True, padding='max_length')
    labels = tokenizer(examples['target_text'], max_length=128, truncation=True, padding='max_length')
    model_inputs['labels'] = labels['input_ids']
    return model_inputs

# 데이터셋 토큰화
tokenized_train_datasets = dataset_train.map(preprocess_function, batched=True)
tokenized_test_datasets = dataset_test.map(preprocess_function, batched=True)

# 학습인자
training_args = TrainingArguments(
    per_device_train_batch_size=batch_size, # 학습 배치 사이즈
    per_device_eval_batch_size=batch_size,  # 평가 배치 사이즈
    output_dir=f't5/{name_folder}',         # 모델 및 체크포인트 저장 디렉토리
    num_train_epochs=train_epochs,          # 학습 에폭 수
    logging_dir=f't5/{name_folder}/logs',   # TensorBoard 로그가 저장될 디렉토리
    logging_steps=100,                      # TensorBoard 로그를 기록할 간격 
    report_to='tensorboard',                # TensorBoard로 로깅
    load_best_model_at_end = True,
    eval_strategy='epoch',
    save_strategy='epoch',                  # 에포크 마다 모델 저장
    # resume_from_checkpoint=True           # 이어 학습
)

# Trainer 객체 생성
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_datasets,
    eval_dataset=tokenized_test_datasets,
    callbacks = [EarlyStoppingCallback(early_stopping_patience=10)]
)

# 모델 학습
trainer.train() # 이어 학습시 (resume_from_checkpoint=checkpoint_dir)

# 모델과 토크나이저 저장
model_save_path = f't5/{name_folder}/model'
tokenizer_save_path = f't5/{name_folder}/model/tokenizer'

model.save_pretrained(model_save_path)
tokenizer.save_pretrained(tokenizer_save_path)

print(f"모델이 '{model_save_path}'에 저장되었습니다.")
print(f"토크나이저가 '{tokenizer_save_path}'에 저장되었습니다.")

/home/tommy/miniconda3/envs/p3/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
TensorFlow installation not found - running with reduced feature set.

NOTE: Using experimental fast data loading logic. To disable, pass
    "--load_fast=false" and report issues on GitHub. More details:
    https://github.com/tensorflow/tensorboard/issues/4784

Serving TensorBoard on localhost; to expose to the network, use a proxy or pass --bind_all
TensorBoard 2.17.1 at http://localhost:1238/ (Press CTRL+C to quit)


TensorBoard 서버가 http://127.0.0.1:1238 에서 실행중.


/home/tommy/miniconda3/envs/p3/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Map: 100%|███████████████████████████████████████████████████████████████████████| 994/994 [00:00<00:00, 11094.57 examples/s]


Epoch,Training Loss,Validation Loss
1,0.286300,0.228090
2,0.226900,0.199032
3,0.205700,0.189071
4,0.190200,0.185347
5,0.181500,0.181025
6,0.176200,0.179272
7,0.169200,0.178308
8,0.165000,0.178412
9,0.157900,0.179132
10,0.155500,0.179456


There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight'].


모델이 't5/transfer_0/model'에 저장되었습니다.
토크나이저가 't5/transfer_0/model/tokenizer'에 저장되었습니다.


### 위의 모델 이서서 계속 훈련

- 얼리 스탑 5321 step
- 이후 이어서 진행 10643 step
- 이후 이어서 진행 

In [6]:
from transformers import T5TokenizerFast, T5ForConditionalGeneration, Trainer, TrainingArguments, EarlyStoppingCallback
from datasets import Dataset
from tqdm import tqdm
import subprocess
import time

name_folder = 'transfer_2_continued'
server_port = '1200'
batch_size = 16
train_epochs = 60

# TensorBoard 서버 실행 (백그라운드)
subprocess.Popen(['tensorboard', f'--logdir=t5/{name_folder}', f'--port={server_port}'])

# TensorBoard 서버가 시작되기를 잠시 기다립니다
time.sleep(5)

# 이후 TensorBoard 서버에 접근할 수 있습니다
print(f"TensorBoard 서버가 http://127.0.0.1:{server_port} 에서 실행중.")

# 모델과 토크나이저 로드
tokenizer = T5TokenizerFast.from_pretrained('paust/pko-t5-base')
model = T5ForConditionalGeneration.from_pretrained('t5/transfer_1_continued/checkpoint-10642/')

# 데이터셋 생성
dataset_train = Dataset.from_dict({'input_text': train_x,'target_text': train_y})
dataset_test = Dataset.from_dict({'input_text': test_x,'target_text': test_y})



# 데이터 전처리 함수 정의
def preprocess_function(examples):
    model_inputs = tokenizer(examples['input_text'], max_length=128, truncation=True, padding='max_length')
    labels = tokenizer(examples['target_text'], max_length=128, truncation=True, padding='max_length')
    model_inputs['labels'] = labels['input_ids']
    return model_inputs

# 데이터셋 토큰화
tokenized_train_datasets = dataset_train.map(preprocess_function, batched=True)
tokenized_test_datasets = dataset_test.map(preprocess_function, batched=True)

# 학습인자
training_args = TrainingArguments(
    per_device_train_batch_size=batch_size, # 학습 배치 사이즈
    per_device_eval_batch_size=batch_size,  # 평가 배치 사이즈
    output_dir=f't5/{name_folder}',         # 모델 및 체크포인트 저장 디렉토리
    num_train_epochs=train_epochs,          # 학습 에폭 수
    logging_dir=f't5/{name_folder}/logs',   # TensorBoard 로그가 저장될 디렉토리
    logging_steps=100,                      # TensorBoard 로그를 기록할 간격 
    report_to='tensorboard',                # TensorBoard로 로깅
    load_best_model_at_end = True,
    eval_strategy='epoch',
    save_strategy='epoch',                  # 에포크 마다 모델 저장
    resume_from_checkpoint=True           # 이어 학습
)

# Trainer 객체 생성
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_datasets,
    eval_dataset=tokenized_test_datasets,
    # callbacks = [EarlyStoppingCallback(early_stopping_patience=100000000)]
)

# 모델 학습
trainer.train() # 이어 학습시 (resume_from_checkpoint=checkpoint_dir)

# 모델과 토크나이저 저장
model_save_path = f't5/{name_folder}/model'
tokenizer_save_path = f't5/{name_folder}/model/tokenizer'

model.save_pretrained(model_save_path)
tokenizer.save_pretrained(tokenizer_save_path)

print(f"모델이 '{model_save_path}'에 저장되었습니다.")
print(f"토크나이저가 '{tokenizer_save_path}'에 저장되었습니다.")

/home/tommy/miniconda3/envs/p3/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
TensorFlow installation not found - running with reduced feature set.

NOTE: Using experimental fast data loading logic. To disable, pass
    "--load_fast=false" and report issues on GitHub. More details:
    https://github.com/tensorflow/tensorboard/issues/4784

Serving TensorBoard on localhost; to expose to the network, use a proxy or pass --bind_all
TensorBoard 2.17.1 at http://localhost:1200/ (Press CTRL+C to quit)


TensorBoard 서버가 http://127.0.0.1:1200 에서 실행중.


/home/tommy/miniconda3/envs/p3/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Map: 100%|████████████████████████████████████████████████████████████████████████| 994/994 [00:00<00:00, 13108.48 examples/s]


Epoch,Training Loss,Validation Loss
1,0.056600,0.311930


모델 계획
데이터 셋 비율 조정 (1 : 5)

  ```py
  # 모델 구성 객체 로드 및 드롭아웃 비율 설정
  config = T5Config.from_pretrained('t5/transfer_1_continued/checkpoint-10642/')
  config.dropout_rate = 0.2  # 드롭아웃 비율을 20%로 설정 (기본값은 0.1)
  
  # 수정된 구성으로 모델 로드
  model = T5ForConditionalGeneration.from_pretrained('t5/transfer_1_continued/checkpoint-10642/', config=config)
  ```

In [6]:
from transformers import T5TokenizerFast, T5ForConditionalGeneration
import torch

# 저장된 모델과 토크나이저 로드
model_save_path = '../model_follow_up/t5/model/'
tokenizer_save_path = '../model_follow_up/t5/model/tokenizer/'

model = T5ForConditionalGeneration.from_pretrained(model_save_path)
tokenizer = T5TokenizerFast.from_pretrained(tokenizer_save_path)


OSError: Incorrect path_or_model_id: '../model_follow_up/t5/model/'. Please provide either the path to a local folder or the repo_id of a model on the Hub.

In [94]:
import torch

# 테스트 입력
test_input = "공포, 건물, 창문, 나무"

# 입력 토큰화
input_ids = tokenizer.encode(test_input, return_tensors='pt')

# GPU가 사용 가능한지 확인하고, 사용 가능한 경우 GPU로 이동
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 모델과 토크나이저를 GPU로 이동
model.to(device)

# 입력 데이터도 GPU로 이동
input_ids = input_ids.to(device)

# 예측을 GPU에서 수행
with torch.no_grad():
    outputs = model.generate(input_ids, max_length=50, num_beams=4, early_stopping=True)

# 예측 결과 디코딩
predicted_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

print(f"테스트 입력: {test_input}")
print(f"모델의 예측: {predicted_text}")

테스트 입력: 공포, 건물, 창문, 나무
모델의 예측: 무서운 분위기가 느껴지는 사진이네요! 😱 창문과 나무가 정말 인상적이에요. 어떤 이야기가 담겨 있을까요? 🌲
